# Builder Agent Configuration & Testing

This notebook configures and tests the Builder Agent using **Snowflake Cortex Complete** (Claude Sonnet) for generating insurance claims with **structured JSON output**.

**Prerequisites:**
- **MUST run both previous notebooks first**:
  - `01_Data_Views_for_Agent_Workflows.ipynb` 
  - `02_Builder_Agent_Development.ipynb`
- Cortex Complete service enabled (Claude Sonnet model)
- All data foundation and configuration tables ready

**What this does:**
- Test Builder Agent using CORTEX.COMPLETE() SQL function
- Generate claims with real patient data and structured JSON output
- Validate JSON parsing and schema compliance
- Test iterative optimization with feedback loops
- Prepare for Insurance Agent integration

**NOTE:** This uses CORTEX.COMPLETE() function calls, not persistent Cortex Agents that appear in Snowflake UI.


## Step 1: Test Data Preparation

Get patient, procedure, and policy data for Builder Agent testing.


In [ ]:
-- Get patient data for Builder Agent test
SELECT 
    PATIENT_ID,
    FIRST_NAME,
    LAST_NAME,
    DATE_OF_BIRTH,
    GENDER,
    MEDICAL_HISTORY_SUMMARY,
    ALLERGIES,
    CURRENT_MEDICATIONS,
    INSURANCE_PROVIDER,
    POLICY_NUMBER,
    GROUP_NUMBER
FROM CLAIMS_DEMO.PUBLIC.PATIENTS 
WHERE PATIENT_ID = 'PAT_001';


In [ ]:
-- Get procedure information and Cigna policy rules
SELECT 
    cp.PROCEDURE_CODE,
    cp.PROCEDURE_NAME,
    cp.MEDICAL_SPECIALTY,
    cp.TYPICAL_COST_RANGE,
    cp.CIGNA_COVERAGE_NOTES,
    cpr.COVERAGE_CRITERIA,
    cpr.PRIOR_AUTH_REQUIRED,
    cpr.MEDICAL_NECESSITY_REQUIREMENTS
FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES cp
LEFT JOIN CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES cpr 
    ON REGEXP_LIKE(cpr.PROCEDURE_CODES, cp.PROCEDURE_CODE)
WHERE cp.PROCEDURE_CODE = '85025';


## Step 2: Create Agent Registry and Actual Agent Functions

Create an agent registry and actual Cortex Agent functions that behave like persistent agents.


In [ ]:
-- Create Agent Registry to track our Cortex Agents
CREATE OR REPLACE TABLE CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY (
    AGENT_ID VARCHAR(50) PRIMARY KEY,
    AGENT_NAME VARCHAR(100),
    AGENT_TYPE VARCHAR(50),
    MODEL_NAME VARCHAR(100),
    SYSTEM_PROMPT TEXT,
    INPUT_SCHEMA TEXT,
    OUTPUT_SCHEMA TEXT,
    STATUS VARCHAR(20),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
-- Create Builder Agent function (using snowflake-arctic model)
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.BUILDER_AGENT(patient_id VARCHAR, procedure_code VARCHAR, clinical_notes VARCHAR)
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        'You are a Builder Agent for Cigna insurance claims. Generate ONLY valid JSON for patient ' || patient_id || ' requesting procedure ' || procedure_code || '. Clinical notes: ' || clinical_notes || '. Use this schema: {"claim_header":{"claim_id":"CLM-2024-001","patient_id":"' || patient_id || '","insurance_provider":"Cigna"},"patient_info":{"first_name":"string","last_name":"string"},"claim_details":{"procedure_codes":["' || procedure_code || '"]},"billing_info":{"total_charges":75.00}}. Return ONLY JSON.'
    )
$$;


In [ ]:
-- Create Insurance Agent function for adversarial validation  
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT(claim_json VARCHAR)
RETURNS VARCHAR  
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        'You are an Insurance Agent for Cigna. Analyze this claim and provide a rebuttal with strength score: ' || claim_json || '. Your goal is to deny claims within policy framework. Generate JSON response: {"rebuttal_summary":"string","denial_reasons":["string"],"strength_score":number,"recommended_improvements":["string"]}. Be strict but fair per Cigna policies. ONLY JSON output.'
    )
$$;


In [ ]:
-- Register both agents in the registry
INSERT INTO CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY (AGENT_ID, AGENT_NAME, AGENT_TYPE, MODEL_NAME, SYSTEM_PROMPT, INPUT_SCHEMA, OUTPUT_SCHEMA, STATUS) VALUES 
('BUILDER_AGENT_V1', 'Cigna Claims Builder Agent', 'CLAIM_GENERATOR', 'snowflake-arctic', 'Builder Agent specialized in creating optimized Cigna insurance claims with JSON output', 'Input: {patient_id, procedure_code, clinical_notes}', 'Output: Complete JSON insurance claim', 'ACTIVE'),
('INSURANCE_AGENT_V1', 'Cigna Insurance Validator Agent', 'CLAIM_VALIDATOR', 'snowflake-arctic', 'Insurance Agent for analyzing claims and providing rebuttals with strength scores', 'Input: {claim_json}', 'Output: JSON with rebuttal and strength score', 'ACTIVE');


## Step 3: Register Builder Agent

Register the Builder Agent in our agent tracking system.


In [ ]:
-- Register Builder Agent in the registry
INSERT INTO CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY (AGENT_ID, AGENT_NAME, AGENT_TYPE, MODEL_NAME, SYSTEM_PROMPT, INPUT_SCHEMA, OUTPUT_SCHEMA, STATUS) VALUES (
    'BUILDER_AGENT_V1',
    'Cigna Claims Builder Agent',
    'CLAIM_GENERATOR', 
    'snowflake-arctic',
    'Builder Agent specialized in creating optimized Cigna insurance claims with structured JSON output',
    'Input: {patient_id, procedure_code, clinical_notes}',
    'Output: Complete JSON insurance claim with all required fields',
    'ACTIVE'
);


## Step 4: Test Builder Agent

Test the Builder Agent function with real patient data.


In [ ]:
-- Test Builder Agent with real patient data
SELECT CLAIMS_DEMO.PUBLIC.BUILDER_AGENT('PAT_001', '85025', 'Annual wellness screening for diabetic patient') as BUILDER_AGENT_OUTPUT;


## Step 5: JSON Validation

Verify Builder Agent output is valid JSON that can be parsed.


In [ ]:
-- Validate JSON output from Builder Agent
WITH agent_test AS (
    SELECT CLAIMS_DEMO.PUBLIC.BUILDER_AGENT('PAT_001', '85025', 'Routine wellness screening') as CLAIM_JSON
)
SELECT 
    CLAIM_JSON,
    TRY_PARSE_JSON(CLAIM_JSON) as PARSED_CLAIM,
    CASE WHEN TRY_PARSE_JSON(CLAIM_JSON) IS NOT NULL THEN 'VALID_JSON' ELSE 'INVALID_JSON' END as VALIDATION_STATUS
FROM agent_test;


## Step 6: Builder Agent Status

Verify Builder Agent is ready for integration with Insurance Agent (Phase 4).


In [ ]:
-- Final verification: Builder Agent ready
SELECT 
    AGENT_ID,
    AGENT_NAME,
    AGENT_TYPE,
    MODEL_NAME,
    STATUS
FROM CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY 
WHERE AGENT_TYPE = 'CLAIM_GENERATOR';

-- Confirm function exists
SELECT 'BUILDER_AGENT' as FUNCTION_NAME, 'ACTIVE' as STATUS;


## Step 2: Builder Agent Testing with Cortex Complete

Test the Builder Agent using Snowflake Cortex Complete SQL function.


In [ ]:
-- Test 1: Simple Builder Agent claim generation  
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    'You are a Builder Agent for Cigna insurance claims. Generate a JSON claim for patient John Smith (PAT_001, DOB 1975-03-15, Male, Diabetes + Hypertension) requesting CBC (85025) for annual wellness screening. Must output ONLY valid JSON with claim_header, patient_info, claim_details, billing_info, supporting_documentation. No additional text.'
) as BUILDER_AGENT_RESPONSE;


## Step 3: Refined Builder Agent with Exact Schema

Test with our exact JSON schema to ensure proper format compliance.


In [ ]:
-- Test 2: Builder Agent with exact schema specification
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    CONCAT(
        'You are a Builder Agent for Cigna claims. Generate ONLY valid JSON following this EXACT schema: ',
        (SELECT JSON_SCHEMA_TEXT FROM CLAIMS_DEMO.PUBLIC.INSURANCE_CLAIM_SCHEMA WHERE SCHEMA_VERSION = 'v1.0'),
        ' Patient: John Smith (PAT_001), DOB 1975-03-15, Male, Diabetes+Hypertension, Cigna CGN123456789, CIGNA_GRP001. Procedure: 85025 CBC for wellness screening. Use today''s date. Include appropriate ICD-10 codes. No additional text - ONLY JSON.'
    )
) as SCHEMA_COMPLIANT_CLAIM;


In [ ]:
-- Test 3: Builder Agent with full database context
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    CONCAT(
        (SELECT SYSTEM_PROMPT FROM CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_CONFIG WHERE CONFIG_ID = 'BUILDER_V1'),
        '\n\nPATIENT: ',
        (SELECT FIRST_NAME || ' ' || LAST_NAME || ', DOB: ' || DATE_OF_BIRTH || ', Gender: ' || GENDER || ', Medical History: ' || MEDICAL_HISTORY_SUMMARY || ', Allergies: ' || ALLERGIES || ', Medications: ' || CURRENT_MEDICATIONS || ', Insurance: ' || INSURANCE_PROVIDER || ' ' || POLICY_NUMBER || ' ' || GROUP_NUMBER FROM CLAIMS_DEMO.PUBLIC.PATIENTS WHERE PATIENT_ID = 'PAT_001'),
        '\n\nPROCEDURE: ',
        (SELECT PROCEDURE_CODE || ' - ' || PROCEDURE_NAME || ' (' || MEDICAL_SPECIALTY || ') Cost: ' || TYPICAL_COST_RANGE || ' Coverage: ' || CIGNA_COVERAGE_NOTES FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES WHERE PROCEDURE_CODE = '85025'),
        '\n\nCLINICAL NOTES: Annual wellness screening for diabetic patient. Patient reports feeling well, good medication compliance.',
        '\n\nJSON SCHEMA: ',
        (SELECT JSON_SCHEMA_TEXT FROM CLAIMS_DEMO.PUBLIC.INSURANCE_CLAIM_SCHEMA WHERE SCHEMA_VERSION = 'v1.0'),
        '\n\nOUTPUT: Return ONLY valid JSON claim. No explanations.'
    )
) as FULL_CONTEXT_CLAIM;


## Step 4: JSON Validation Testing

Verify that Builder Agent output can be properly parsed as JSON for frontend integration.


In [ ]:
-- Test JSON parsing and validation
WITH builder_output AS (
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'claude-4-sonnet',
        'You are a Builder Agent for Cigna. Generate ONLY valid JSON for: Patient John Smith (PAT_001, Male, 1975-03-15, Diabetes+Hypertension, Cigna CGN123456789) requesting CBC (85025) for wellness screening. Follow schema exactly: {"claim_header":{"claim_id":"CLM-YYYY-XXXXXX","patient_id":"PAT_001","insurance_provider":"Cigna"},"patient_info":{"first_name":"John","last_name":"Smith"},"claim_details":{"procedure_codes":["85025"]},"billing_info":{"total_charges":75.00}}. No extra text.'
    ) as JSON_CLAIM
)
SELECT 
    JSON_CLAIM,
    TRY_PARSE_JSON(JSON_CLAIM) as PARSED_JSON,
    CASE WHEN TRY_PARSE_JSON(JSON_CLAIM) IS NOT NULL THEN 'VALID_JSON' ELSE 'INVALID_JSON' END as JSON_STATUS
FROM builder_output;


## Step 5: Feedback Loop Testing

Test Builder Agent's ability to optimize claims based on Insurance Agent rebuttal feedback.


In [ ]:
-- Store initial Builder Agent output for feedback testing
INSERT INTO CLAIMS_DEMO.PUBLIC.AGENT_INTERACTIONS VALUES (
    'INT_TEST_001',
    'SESSION_TEST_001', 
    'BUILDER',
    PARSE_JSON('{"patient_id": "PAT_001", "procedure_code": "85025", "clinical_notes": "Annual wellness screening"}'),
    PARSE_JSON('{"claim_header":{"claim_id":"CLM-2024-001","patient_id":"PAT_001","insurance_provider":"Cigna"},"claim_details":{"procedure_codes":["85025"]},"billing_info":{"total_charges":75.00}}'),
    CURRENT_TIMESTAMP(),
    'SUCCESS',
    NULL
);


In [ ]:
-- Test Builder Agent optimization after simulated rebuttal
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    CONCAT(
        (SELECT PROMPT_TEXT FROM CLAIMS_DEMO.PUBLIC.CLAIM_GENERATION_PROMPTS WHERE PROMPT_ID = 'REBUTTAL_RESPONSE'),
        '\n\nORIGINAL CLAIM: {"claim_header":{"claim_id":"CLM-2024-001","patient_id":"PAT_001"},"claim_details":{"procedure_codes":["85025"]}}',
        '\n\nINSURANCE AGENT REBUTTAL: CLAIM DENIED - Medical necessity not established. Insufficient documentation for preventive CBC in diabetic patient. Please provide physician notes justifying medical necessity per Cigna policy section 4.1.',
        '\n\nSTRENGTH SCORE: 25/100',
        '\n\nOUTPUT: Return ONLY improved JSON claim addressing the medical necessity issue.'
    )
) as OPTIMIZED_CLAIM;


## Step 6: Builder Agent Function Creation

Create a reusable function that encapsulates the complete Builder Agent workflow.


In [ ]:
-- Create Builder Agent workflow function using Claude Sonnet
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_GENERATE_CLAIM(
    patient_id VARCHAR,
    procedure_code VARCHAR,
    clinical_notes VARCHAR
)
RETURNS VARIANT
LANGUAGE SQL
AS
$$
    SELECT PARSE_JSON(
        SNOWFLAKE.CORTEX.COMPLETE(
            'claude-4-sonnet',
            CONCAT(
                (SELECT SYSTEM_PROMPT FROM CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_CONFIG WHERE CONFIG_ID = 'BUILDER_V1'),
                '\n\nPATIENT: ',
                (SELECT FIRST_NAME || ' ' || LAST_NAME || ', DOB: ' || DATE_OF_BIRTH || ', Gender: ' || GENDER || ', Medical History: ' || MEDICAL_HISTORY_SUMMARY || ', Allergies: ' || ALLERGIES || ', Medications: ' || CURRENT_MEDICATIONS || ', Insurance: ' || INSURANCE_PROVIDER || ' ' || POLICY_NUMBER || ' ' || GROUP_NUMBER FROM CLAIMS_DEMO.PUBLIC.PATIENTS WHERE PATIENT_ID = patient_id),
                '\n\nPROCEDURE: ',
                (SELECT PROCEDURE_CODE || ' - ' || PROCEDURE_NAME || ' (' || MEDICAL_SPECIALTY || ') Cost: ' || TYPICAL_COST_RANGE || ' Coverage: ' || CIGNA_COVERAGE_NOTES FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES WHERE PROCEDURE_CODE = procedure_code),
                '\n\nCLINICAL NOTES: ', clinical_notes,
                '\n\nJSON SCHEMA: ',
                (SELECT JSON_SCHEMA_TEXT FROM CLAIMS_DEMO.PUBLIC.INSURANCE_CLAIM_SCHEMA WHERE SCHEMA_VERSION = 'v1.0'),
                '\n\nOUTPUT: Return ONLY valid JSON claim. No explanations.'
            )
        )
    )
$$;


## Step 7: End-to-End Workflow Test

Test the complete Provider → Builder Agent → Insurance Agent → Optimization workflow.


In [ ]:
-- Test complete workflow: Generate claim using Builder Agent with Claude Sonnet
SELECT 
    'PAT_001' as PATIENT_ID,
    '85025' as PROCEDURE_CODE,
    'Annual wellness screening - patient requests routine labs' as CLINICAL_NOTES,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-4-sonnet', 
        CONCAT(
            'Builder Agent for Cigna: Generate JSON claim for John Smith (PAT_001, 1975-03-15, Male, Diabetes+Hypertension, Cigna CGN123456789) requesting CBC (85025). Clinical: Annual wellness screening. Must follow JSON schema with claim_header, patient_info, claim_details, billing_info, supporting_documentation. ONLY JSON output.'
        )
    ) as GENERATED_CLAIM;


## Step 8: Verification & Next Steps

Verify Builder Agent configuration is complete and ready for Insurance Agent integration.


In [ ]:
-- Final verification: Builder Agent capabilities confirmed
SELECT 
    'BUILDER_AGENT_CONFIGURED' as STATUS,
    'JSON_OUTPUT_VALIDATED' as JSON_STATUS,
    'OPTIMIZATION_LOOP_TESTED' as FEEDBACK_STATUS,
    'READY_FOR_INSURANCE_AGENT' as NEXT_PHASE;

-- Test different patient scenarios  
SELECT 
    PATIENT_ID,
    FIRST_NAME || ' ' || LAST_NAME as PATIENT_NAME,
    MEDICAL_HISTORY_SUMMARY,
    'Ready for Builder Agent claim generation' as STATUS
FROM CLAIMS_DEMO.PUBLIC.PATIENTS
ORDER BY PATIENT_ID;
